In [1]:
import torch
import numpy as np
from PIL import Image, ImageDraw, ImageFont
from facenet_pytorch import MTCNN, InceptionResnetV1
import os

# Load model and embeddings
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
mtcnn = MTCNN(image_size=160, margin=0, keep_all=True, device=device)
resnet = InceptionResnetV1(pretrained='vggface2').eval().to(device)

# Student embeddings loading
student_data = np.load("student_embeddings_02.npy", allow_pickle=True)
student_embeddings = student_data.item()

# Group image loading
group_img_path = "picnic01.jpg"  # replace with your image path
img = Image.open(group_img_path).convert('RGB')

# Detect faces
boxes, probs = mtcnn.detect(img)

if boxes is None:
    print("❌ No faces detected.")
else:
    print(f"✅ Detected {len(boxes)} face(s).")

present_students = []

# Drawing object
draw = ImageDraw.Draw(img)
font = ImageFont.load_default()

# Processing each detected face
for i, box in enumerate(boxes):
    x1, y1, x2, y2 = map(int, box)
    face_crop = img.crop((x1, y1, x2, y2)).resize((160, 160))
    face_tensor = torch.tensor(np.array(face_crop)).permute(2, 0, 1).unsqueeze(0).float().to(device) / 255.0

    with torch.no_grad():
        embedding = resnet(face_tensor)

    # Comparing with all stored embeddings
    matched = False
    for reg_num, emb in student_embeddings.items():
        emb_tensor = torch.tensor(emb).unsqueeze(0).to(device).float()
        dist = torch.norm(embedding - emb_tensor).item()
        print(f"🧪 Comparing with {reg_num}: distance = {dist:.4f}")

        if dist < 0.9:  # Adjust threshold if needed
            print(f"✅ Match found: {reg_num}")
            draw.rectangle([x1, y1, x2, y2], outline="green", width=2)
            draw.text((x1, y2 + 2), reg_num, fill="green", font=font)
            present_students.append(reg_num)
            matched = True
            break

    if not matched:
        print(f"⚠️ No match for face #{i + 1}")
        draw.rectangle([x1, y1, x2, y2], outline="red", width=2)
        draw.text((x1, y2 + 2), "Unknown", fill="red", font=font)


img.save("output_with_boxes.jpg")
img.show()

print("\n✅ Present Students:")
print(list(set(present_students)))


✅ Detected 21 face(s).
🧪 Comparing with 2241013015: distance = 1.0787
🧪 Comparing with 22410160124: distance = 1.1343
🧪 Comparing with 2241016042: distance = 0.5991
✅ Match found: 2241016042
🧪 Comparing with 2241013015: distance = 1.2990
🧪 Comparing with 22410160124: distance = 1.1951
🧪 Comparing with 2241016042: distance = 0.9828
🧪 Comparing with 2241016056: distance = 1.0470
🧪 Comparing with 2241016078: distance = 1.1022
🧪 Comparing with 2241016088: distance = 0.8192
✅ Match found: 2241016088
🧪 Comparing with 2241013015: distance = 1.1686
🧪 Comparing with 22410160124: distance = 1.0066
🧪 Comparing with 2241016042: distance = 1.0131
🧪 Comparing with 2241016056: distance = 1.1401
🧪 Comparing with 2241016078: distance = 0.8659
✅ Match found: 2241016078
🧪 Comparing with 2241013015: distance = 1.2133
🧪 Comparing with 22410160124: distance = 0.9282
🧪 Comparing with 2241016042: distance = 0.9247
🧪 Comparing with 2241016056: distance = 0.8729
✅ Match found: 2241016056
🧪 Comparing with 224101